# Constraint Docking

This feature is available since version 2.0.0

In [1]:
import tempfile, os, requests, gzip, shutil, yaml
from multiprocessing import cpu_count
from moldrug.data import ligands, boxes, receptor_pdbqt, receptor_pdb, constraintref
from moldrug import utils
tmp_path = tempfile.TemporaryDirectory()
import numpy as np

In [2]:
# Setting the working directory (you could change it but it MUST be an absolute path)
wd = tmp_path.name
# os.makedirs('wd_tutorial', exist_ok=True)
# wd = os.path.abspath('wd_tutorial')


In [3]:
# Getting the data
lig = ligands.r_x0161
box = boxes.r_x0161
with open(os.path.join(wd, 'x0161.pdbqt'), 'w') as f:
    f.write(receptor_pdbqt.r_x0161)
with open(os.path.join(wd, 'x0161.pdb'), 'w') as f:
    f.write(receptor_pdb.r_x0161)
with open(os.path.join(wd, 'ref.sdf'), 'w') as f:
    f.write(constraintref.r_x0161)
# Getting the CReM data base
url = "http://www.qsar4u.com/files/cremdb/replacements02_sc2.db.gz"
r = requests.get(url, allow_redirects=True)
crem_dbgz_path = os.path.join(wd,'crem.db.gz')
crem_db_path = os.path.join(wd,'crem.db')
open(crem_dbgz_path, 'wb').write(r.content)
with gzip.open(crem_dbgz_path, 'rb') as f_in:
    with open(crem_db_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(lig)
print(box)
print(wd, os.listdir(wd))

COC(=O)C=1C=CC(=CC1)S(=O)(=O)N
{'A': {'boxcenter': [12.11, 1.84, 23.56], 'boxsize': [22.5, 22.5, 22.5]}}
/tmp/tmpaxc3g0le ['x0161.pdb', 'ref.sdf', 'crem.db.gz', 'x0161.pdbqt', 'crem.db']


## Setting the configuration

In [4]:
config ={
    "01_grow": {
        "type": "GA",
        "njobs": 6,
        "seed_mol": lig,
        "AddHs": True,
        "costfunc": "Cost",
        "costfunc_kwargs": {
            "vina_executable": "vina",
            "receptor_pdbqt_path": os.path.join(wd, 'x0161.pdbqt'),
            "boxcenter": box['A']['boxcenter'],
            "boxsize": box['A']['boxsize'],
            "exhaustiveness": 4,
            "ncores": int(cpu_count() / 6),
            "num_modes": 1,
            "constraint": True,
            "constraint_type": "score_only", # local_only
            "constraint_ref": os.path.join(wd, 'ref.sdf'),
            "constraint_receptor_pdb_path": os.path.join(wd, 'x0161.pdb'),
            "constraint_num_conf": 100,
            "constraint_minimum_conf_rms": 0.01
        },
        "crem_db_path": crem_db_path,
        "maxiter": 5,
        "popsize": 25,
        "beta": 0.001,
        "pc": 1,
        "get_similar": False,
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 0,
            "max_size": 0,
            "min_inc": -5,
            "max_inc": 6,
            "ncores": 12
        },
        "save_pop_every_gen": 10,
        "deffnm": "01_grow"
    },
    "02_allow_grow": {
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 0,
            "max_size": 2,
            "min_inc": -5,
            "max_inc": 3,
            "ncores": 12
        },
        "maxiter": 5,
        "deffnm": "02_allow_grow"
    },
    "03_pure_mutate": {
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 1,
            "max_size": 8,
            "min_inc": -5,
            "max_inc": 3,
            "ncores": 12
        },
        "maxiter": 2,
        "deffnm": "03_pure_mutate"
    },
    "04_local": {
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 0,
            "max_size": 1,
            "min_inc": -1,
            "max_inc": 1,
            "ncores": 12
        },
        "maxiter": 2,
        "deffnm": "04_local"
    }
}
# Save the config as a yaml file
with open(os.path.join(wd, 'config.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(wd)

['x0161.pdb', 'ref.sdf', 'crem.db.gz', 'x0161.pdbqt', 'config.yml', 'crem.db']

## Move to the directory and run moldrug

In [5]:
cwd = os.getcwd()
os.chdir(wd)
! moldrug config.yml
os.chdir(cwd)
os.listdir(wd)
os.chdir(cwd)

You are using moldrug: 2.0.0.

The main job is being executed.


Creating the first population with 25 members:
100%|███████████████████████████████████████████| 25/25 [00:19<00:00,  1.30it/s]
Initial Population: Best individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0)
File 01_grow_pop.sdf was createad!
Evaluating generation 1 / 5:
100%|███████████████████████████████████████████| 25/25 [00:31<00:00,  1.27s/it]
Generation 1: Best Individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0).

Evaluating generation 2 / 5:
100%|███████████████████████████████████████████| 25/25 [00:30<00:00,  1.23s/it]
Generation 2: Best Individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0).

Evaluating generation 3 / 5:
100%|███████████████████████████████████████████| 25/25 [00:30<00:00,  1.20s/it]
Generation 3: Best Individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0).

Evaluating generation 4 / 

In [6]:
os.listdir(wd)

['x0161.pdb',
 '01_grow_pop.sdf',
 '04_local_result.pbz2',
 'ref.sdf',
 '02_allow_grow_pop.pbz2',
 '03_pure_mutate_result.pbz2',
 'crem.db.gz',
 '03_pure_mutate_pop.sdf',
 '04_local_pop.pbz2',
 '01_grow_result.pbz2',
 '02_allow_grow_pop.sdf',
 '03_pure_mutate_pop.pbz2',
 '02_allow_grow_result.pbz2',
 '04_local_pop.sdf',
 'x0161.pdbqt',
 'config.yml',
 'crem.db',
 '01_grow_pop.pbz2']

## Visualization

If needed install the packages through pip:

`! pip install nglview, parmed`


In [7]:
import nglview as nv
import parmed as pmd
from ipywidgets import IntSlider, VBox

In [8]:
parms = pmd.rdkit.from_sdf(os.path.join(wd, '04_local_pop.sdf')) # require parmed, rdkit
parms

[<Structure 23 atoms; 1 residues; 23 bonds; NOT parametrized>,
 <Structure 35 atoms; 1 residues; 35 bonds; NOT parametrized>,
 <Structure 35 atoms; 1 residues; 35 bonds; NOT parametrized>,
 <Structure 23 atoms; 1 residues; 23 bonds; NOT parametrized>,
 <Structure 34 atoms; 1 residues; 34 bonds; NOT parametrized>,
 <Structure 33 atoms; 1 residues; 33 bonds; NOT parametrized>,
 <Structure 30 atoms; 1 residues; 31 bonds; NOT parametrized>,
 <Structure 26 atoms; 1 residues; 26 bonds; NOT parametrized>,
 <Structure 25 atoms; 1 residues; 25 bonds; NOT parametrized>,
 <Structure 29 atoms; 1 residues; 29 bonds; NOT parametrized>,
 <Structure 30 atoms; 1 residues; 30 bonds; NOT parametrized>,
 <Structure 35 atoms; 1 residues; 35 bonds; NOT parametrized>,
 <Structure 30 atoms; 1 residues; 31 bonds; NOT parametrized>,
 <Structure 23 atoms; 1 residues; 23 bonds; NOT parametrized>,
 <Structure 27 atoms; 1 residues; 27 bonds; NOT parametrized>,
 <Structure 34 atoms; 1 residues; 34 bonds; NOT paramet

In [9]:
view = nv.NGLWidget()

slider = IntSlider(max=len(parms)-1)

def show_one_ligand(change):
    val = change['new']
    view.show_only([val])
    
slider.observe(show_one_ligand, 'value')

VBox([view, slider])

In [10]:
for p in parms:
    view.add_structure(nv.ParmEdTrajectory(p))
view.show_only([0])

In [11]:
result = utils.decompress_pickle(os.path.join(wd, '04_local_result.pbz2'))
valid_individuals_vina_score = [individual.vina_score for individual in result.pop if individual.vina_score != np.inf]
print(f"avg = {np.average(valid_individuals_vina_score)}")
print(f"min = {min(valid_individuals_vina_score)}")
print(f"max = {max(valid_individuals_vina_score)}")

avg = -2.5396768
min = -4.45914
max = 3.74655


## See how it looks with local_only

In [12]:
# Save the new config as a yaml file
new_config = config.copy()
new_config['01_grow']['costfunc_kwargs']['constraint_type'] = 'local_only'

with open(os.path.join(wd, 'new_config.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(wd)

['x0161.pdb',
 '01_grow_pop.sdf',
 '04_local_result.pbz2',
 'ref.sdf',
 '02_allow_grow_pop.pbz2',
 '03_pure_mutate_result.pbz2',
 'crem.db.gz',
 '03_pure_mutate_pop.sdf',
 '04_local_pop.pbz2',
 '01_grow_result.pbz2',
 '02_allow_grow_pop.sdf',
 '03_pure_mutate_pop.pbz2',
 '02_allow_grow_result.pbz2',
 '04_local_pop.sdf',
 'new_config.yml',
 'x0161.pdbqt',
 'config.yml',
 'crem.db',
 '01_grow_pop.pbz2']

## Move back to the directory and run moldrug

In [13]:
cwd = os.getcwd()
os.chdir(wd)
! moldrug new_config.yml
os.chdir(cwd)
os.listdir(wd)
os.chdir(cwd)

You are using moldrug: 2.0.0.

The main job is being executed.


Creating the first population with 25 members:
100%|███████████████████████████████████████████| 25/25 [00:23<00:00,  1.06it/s]
Initial Population: Best individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0)
Individual(idx = 10, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1Sc1ccsc1, cost = 1.0) does not have a valid pdbqt.
File 01_grow_pop.sdf was createad!
Evaluating generation 1 / 5:
100%|███████████████████████████████████████████| 25/25 [00:33<00:00,  1.36s/it]
Generation 1: Best Individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0).

Evaluating generation 2 / 5:
100%|███████████████████████████████████████████| 25/25 [00:32<00:00,  1.32s/it]
Generation 2: Best Individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0).

Evaluating generation 3 / 5:
100%|███████████████████████████████████████████| 25/25 [00:32<00:00,  1.30s/it]
Generation 3: Best In

In [14]:
parms = pmd.rdkit.from_sdf(os.path.join(wd, '04_local_pop.sdf')) # require parmed, rdkit
parms

[<Structure 29 atoms; 1 residues; 30 bonds; NOT parametrized>,
 <Structure 39 atoms; 1 residues; 40 bonds; NOT parametrized>,
 <Structure 29 atoms; 1 residues; 30 bonds; NOT parametrized>,
 <Structure 39 atoms; 1 residues; 40 bonds; NOT parametrized>,
 <Structure 41 atoms; 1 residues; 42 bonds; NOT parametrized>,
 <Structure 41 atoms; 1 residues; 42 bonds; NOT parametrized>,
 <Structure 41 atoms; 1 residues; 42 bonds; NOT parametrized>,
 <Structure 43 atoms; 1 residues; 44 bonds; NOT parametrized>,
 <Structure 42 atoms; 1 residues; 43 bonds; NOT parametrized>,
 <Structure 46 atoms; 1 residues; 47 bonds; NOT parametrized>,
 <Structure 23 atoms; 1 residues; 23 bonds; NOT parametrized>,
 <Structure 23 atoms; 1 residues; 23 bonds; NOT parametrized>,
 <Structure 39 atoms; 1 residues; 39 bonds; NOT parametrized>,
 <Structure 30 atoms; 1 residues; 30 bonds; NOT parametrized>,
 <Structure 29 atoms; 1 residues; 29 bonds; NOT parametrized>,
 <Structure 32 atoms; 1 residues; 33 bonds; NOT paramet

In [15]:
view = nv.NGLWidget()

slider = IntSlider(max=len(parms)-1)

def show_one_ligand(change):
    val = change['new']
    view.show_only([val])
    
slider.observe(show_one_ligand, 'value')

VBox([view, slider])

In [16]:
for p in parms:
    view.add_structure(nv.ParmEdTrajectory(p))
view.show_only([0])

In [17]:
result = utils.decompress_pickle(os.path.join(wd, '04_local_result.pbz2'))
valid_individuals_vina_score = [individual.vina_score for individual in result.pop if individual.vina_score != np.inf]
print(f"avg = {np.average(valid_individuals_vina_score)}")
print(f"min = {min(valid_individuals_vina_score)}")
print(f"max = {max(valid_individuals_vina_score)}")


avg = -5.428390416666667
min = -6.44023
max = -4.33338


As you can see now the fix core change a little. Could be that (when you run the tutorial by yourself) MolDrug can not optimize the cost function. One of the main cause of this behavior is the desirability definition which is for vina_score:

```
'vina_score': {
    'w': 1,
    'SmallerTheBest': {
        'Target': -12,
        'UpperLimit': -6,
        'r': 1
    }
}
```

This means that if the vina score is not lower than -6 the cost function will be always 1. In this scenario all were are not optimizing over the generations the "best" individuals. In other words, it is important to see what is the behavior of our system (one small simulation) and then tune your parameters accordantly. On possibility for this system is change UpperLimit to, for example -4. There are a lot of other different options that you could play around (e.g. increase the generations, etc...). It is important to remark that this example is just for the tutorial, in a real project you may need to increase generations, tune the crem keywords, etc. I let it to you as a challenge. Happy simulations!

Just as a bonus I let you here how to change the desirability definition.

## Change the desirability

In [18]:
# Save the new config as a yaml file
tune_config = new_config.copy()
tune_config['01_grow']['costfunc_kwargs']['desirability'] = {
        'qed': {
            'w': 1,
            'LargerTheBest': {
                'LowerLimit': 0.1,
                'Target': 0.75,
                'r': 1
            }
        },
        'sa_score': {
            'w': 1,
            'SmallerTheBest': {
                'Target': 3,
                'UpperLimit': 7,
                'r': 1
            }
        },
        'vina_score': {
            'w': 1,
            'SmallerTheBest': {
                'Target': -8,
                'UpperLimit': -3,
                'r': 1
            }
        }
    }

with open(os.path.join(wd, 'tune_config.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(wd)


['x0161.pdb',
 '01_grow_pop.sdf',
 '04_local_result.pbz2',
 'ref.sdf',
 '02_allow_grow_pop.pbz2',
 '03_pure_mutate_result.pbz2',
 'crem.db.gz',
 'tune_config.yml',
 '03_pure_mutate_pop.sdf',
 '04_local_pop.pbz2',
 '01_grow_result.pbz2',
 '02_allow_grow_pop.sdf',
 '03_pure_mutate_pop.pbz2',
 '02_allow_grow_result.pbz2',
 '04_local_pop.sdf',
 'new_config.yml',
 'x0161.pdbqt',
 'config.yml',
 'crem.db',
 '01_grow_pop.pbz2']

In [19]:

cwd = os.getcwd()
os.chdir(wd)
! moldrug tune_config.yml
os.chdir(cwd)
os.listdir(wd)
os.chdir(cwd)

You are using moldrug: 2.0.0.

The main job is being executed.


Creating the first population with 25 members:
100%|███████████████████████████████████████████| 25/25 [00:18<00:00,  1.32it/s]
Initial Population: Best individual: Individual(idx = 4, smiles = COC(=O)c1ccc(S(N)(=O)=O)c(Cl)c1, cost = 0.5458783343354592)
File 01_grow_pop.sdf was createad!
Evaluating generation 1 / 5:
100%|███████████████████████████████████████████| 25/25 [00:31<00:00,  1.26s/it]
Generation 1: Best Individual: Individual(idx = 39, smiles = COCCOCC(O)COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 0.5061900814545027).

Evaluating generation 2 / 5:
100%|███████████████████████████████████████████| 25/25 [00:40<00:00,  1.60s/it]
Generation 2: Best Individual: Individual(idx = 39, smiles = COCCOCC(O)COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 0.5061900814545027).

Evaluating generation 3 / 5:
100%|███████████████████████████████████████████| 25/25 [00:41<00:00,  1.68s/it]
Generation 3: Best Individual: Individual(idx = 39, smiles

In [20]:
result = utils.decompress_pickle(os.path.join(wd, '04_local_result.pbz2'))
valid_individuals_vina_score = [individual.vina_score for individual in result.pop if individual.vina_score != np.inf]
print(f"avg = {np.average(valid_individuals_vina_score)}")
print(f"min = {min(valid_individuals_vina_score)}")
print(f"max = {max(valid_individuals_vina_score)}")


avg = -5.079627200000001
min = -5.98645
max = -4.2378


It was not to much improvement on this simulation (maybe in yours the difference is bigger)